<a href="https://colab.research.google.com/github/piyush0511/SpellChecker-AutoCorrect/blob/main/Autocorrect-AttentionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from tensorflow import keras
import random
import string
import pandas as pd
from tqdm import tqdm
import pickle

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

Found GPU at: /device:GPU:0


In [2]:
# Download the file
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

In [3]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                 if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z)
  w = re.sub(r"[^a-zA-Z ]+", "", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  return w

In [4]:
en_sentence = u"May I borrow this book?"
print(preprocess_sentence(en_sentence))

may i borrow this book


In [5]:
def noise(data):
  input_texts = []
  target_texts = []
  for line in data:
    input_text = line.lower()
    input_text = re.sub(r'[^a-zA-Z ]+', '', input_text)
    target_text = "\t" + input_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    inp = input_text
    for _ in range(2):
      input_text = inp
      for i in range(np.random.choice(np.arange(0, 2), p=[0.1, 0.9])):

          input_text = input_text.replace(random.choice(list(input_text)),random.choice(string.ascii_letters))

      #input_text = "\t" + input_text + "\n"
      input_texts.append(input_text.lower())
      target_texts.append(target_text)

  return (input_texts, target_texts)


In [6]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  en = [preprocess_sentence(line.split('\t')[0]) for line in lines[:num_examples]]

  inp, targ = noise(en)

  return inp, targ

In [7]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', char_level = True)
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [8]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  inp_lang, targ_lang = create_dataset(path, num_examples)
  print(inp_lang[55000])
  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### Limit the size of the dataset to experiment faster (optional)

Training on the complete dataset of >100,000 sentences will take a long time. To train faster, we can limit the size of the dataset to 30,000 sentences (of course, translation quality degrades with fewer data):

In [9]:
# Try experimenting with the size of that dataset
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file,
                                                                num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

tom was frigotened


In [10]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

240000 240000 60000 60000


In [11]:
def convert(lang, tensor):
  for t in tensor:
    if t != 0:
      print(f'{t} ----> {lang.index_word[t]}')

In [12]:
inp_lang.word_index

{' ': 1,
 'a': 5,
 'b': 21,
 'c': 18,
 'd': 12,
 'e': 2,
 'f': 19,
 'g': 17,
 'h': 9,
 'i': 6,
 'j': 24,
 'k': 22,
 'l': 11,
 'm': 13,
 'n': 8,
 'o': 4,
 'p': 20,
 'q': 26,
 'r': 10,
 's': 7,
 't': 3,
 'u': 15,
 'v': 23,
 'w': 16,
 'x': 25,
 'y': 14,
 'z': 27}

In [13]:
print("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[2])
print()
print("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[2])

Input Language; index to word mapping
3 ----> t
4 ----> o
13 ----> m
1 ---->  
2 ----> e
25 ----> x
20 ----> p
11 ----> l
5 ----> a
6 ----> i
8 ----> n
2 ----> e
12 ----> d
1 ---->  
3 ----> t
9 ----> h
2 ----> e
1 ---->  
10 ----> r
2 ----> e
5 ----> a
25 ----> x
4 ----> o
8 ----> n
25 ----> x
1 ---->  
19 ----> f
4 ----> o
10 ----> r
1 ---->  
9 ----> h
6 ----> i
25 ----> x
1 ---->  
12 ----> d
2 ----> e
18 ----> c
6 ----> i
25 ----> x
6 ----> i
4 ----> o
8 ----> n

Target Language; index to word mapping
11 ----> 	
3 ----> t
4 ----> o
15 ----> m
1 ---->  
2 ----> e
27 ----> x
22 ----> p
13 ----> l
5 ----> a
6 ----> i
8 ----> n
2 ----> e
14 ----> d
1 ---->  
3 ----> t
9 ----> h
2 ----> e
1 ---->  
10 ----> r
2 ----> e
5 ----> a
7 ----> s
4 ----> o
8 ----> n
7 ----> s
1 ---->  
21 ----> f
4 ----> o
10 ----> r
1 ---->  
9 ----> h
6 ----> i
7 ----> s
1 ---->  
14 ----> d
2 ----> e
20 ----> c
6 ----> i
7 ----> s
6 ----> i
4 ----> o
8 ----> n
12 ----> 



### Create a tf.data dataset

In [14]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [15]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 42]), TensorShape([64, 44]))

In [16]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [17]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print('Encoder output shape: (batch size, sequence length, units)', sample_output.shape)
print('Encoder Hidden state shape: (batch size, units)', sample_hidden.shape)

Encoder output shape: (batch size, sequence length, units) (64, 42, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [18]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [19]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units)", attention_result.shape)
print("Attention weights shape: (batch_size, sequence_length, 1)", attention_weights.shape)

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 42, 1)


In [20]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [21]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print('Decoder output shape: (batch_size, vocab size)', sample_decoder_output.shape)

Decoder output shape: (batch_size, vocab size) (64, 30)


## Define the optimizer and the loss function

In [22]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [23]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [24]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['\t']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [25]:
EPOCHS = 2

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')
  # saving (checkpoint) the model every 2 epochs
  #if (epoch + 1) % 2 == 0:
  checkpoint.save(file_prefix=checkpoint_prefix)

  print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f}')
  print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')

Epoch 1 Batch 0 Loss 2.1934
Epoch 1 Batch 100 Loss 1.4314
Epoch 1 Batch 200 Loss 1.2208
Epoch 1 Batch 300 Loss 1.4231
Epoch 1 Batch 400 Loss 1.1502
Epoch 1 Batch 500 Loss 1.1012
Epoch 1 Batch 600 Loss 0.9260
Epoch 1 Batch 700 Loss 0.2288
Epoch 1 Batch 800 Loss 1.5971
Epoch 1 Batch 900 Loss 0.2361
Epoch 1 Batch 1000 Loss 0.2600
Epoch 1 Batch 1100 Loss 0.2416
Epoch 1 Batch 1200 Loss 1.3851
Epoch 1 Batch 1300 Loss 1.3081
Epoch 1 Batch 1400 Loss 1.2971
Epoch 1 Batch 1500 Loss 1.2592
Epoch 1 Batch 1600 Loss 1.2324
Epoch 1 Batch 1700 Loss 1.2861
Epoch 1 Batch 1800 Loss 1.2621
Epoch 1 Batch 1900 Loss 1.1787
Epoch 1 Batch 2000 Loss 1.2171
Epoch 1 Batch 2100 Loss 1.1915
Epoch 1 Batch 2200 Loss 1.1771
Epoch 1 Batch 2300 Loss 1.2517
Epoch 1 Batch 2400 Loss 1.1051
Epoch 1 Batch 2500 Loss 1.1752
Epoch 1 Batch 2600 Loss 1.1929
Epoch 1 Batch 2700 Loss 1.1849
Epoch 1 Batch 2800 Loss 1.0898
Epoch 1 Batch 2900 Loss 0.9172
Epoch 1 Batch 3000 Loss 0.2090
Epoch 1 Batch 3100 Loss 0.2665
Epoch 1 Batch 3200 L

## Auto Correct

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [26]:
assert False

AssertionError: ignored

In [27]:
def evaluate(sentence):

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['\t']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)


    attention_weights = tf.reshape(attention_weights, (-1, ))

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ''

    if targ_lang.index_word[predicted_id] == '\n':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [28]:
def autocorrect(sentence):
  result, sentence = evaluate(sentence)

  print('Input:', sentence)
  print('Predicted translation:', result)

## Restore the latest checkpoint and test

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [30]:
autocorrect(u'im just watching')

Input: im just watching
Predicted translation: im the the the the the the the the the the t
